In [36]:
import os

In [37]:
current_path=os.getcwd()
current_path

'c:\\Users\\Asus\\Desktop\\all projects & learnings\\llm-rag-fastapi-study'

In [38]:
folder_path="C:\\Users\\Asus\\Documents\\all_doc"

In [39]:
from langchain_community.document_loaders import PyPDFLoader

In [40]:
all_doc=[]
for file in os.listdir(folder_path):
    if file.endswith('.pdf'):
        loader=PyPDFLoader(os.path.join(folder_path+"\\"+file))
        doc= loader.load()
        all_doc.extend(doc)
all_doc

[Document(metadata={'producer': 'iLovePDF', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-09-30T18:26:34+05:30', 'author': 'R, Nivash', 'moddate': '2025-09-30T12:59:10+00:00', 'source': 'C:\\Users\\Asus\\Documents\\all_doc\\NIVASH__R.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='NIVASH R \n    Coimbatore |         nivash2920@gmail.com | ✆ +91 94452 91158 |  nivash-r \n \nSUMMARY \nInnovative Software Engineer with a strong foundation in designing, optimizing, and automating data -driven workflows. \nRecognized for bridging data engineering with emerging AI practices to create intelligent, end-to-end systems that drive \nbusiness value. Skilled at transforming complex data into meaningful insights and ensuring that technical solutions align \nwith organizational goals in a fast-evolving technological landscape. Committed to delivering scalable and reliable \nsolutions that drive efficiency, strengthen workflows, and keep pace with rapidly changing busin

In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [42]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=10)
chunk=splitter.split_documents(all_doc)
print("splitted into ",len(chunk),"chunks")

splitted into  15 chunks


In [43]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [44]:
embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [45]:
from langchain_community.vectorstores import FAISS

In [46]:
db=FAISS.from_documents(chunk,embedding)

In [47]:
print("uploaded ",db.index.ntotal,"chunk to vector db")

uploaded  15 chunk to vector db


In [48]:
from langchain_community.llms import Ollama

In [49]:
llm = Ollama(model="llama3.2")

C:\Users\Asus\AppData\Local\Temp\ipykernel_14564\1555928118.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [50]:
retrieve= db.as_retriever(search_kwargs={'k':10})

In [51]:
from langchain_core.prompts import ChatPromptTemplate 

In [52]:
template="""Give response only from the below details
{context}
question : {question}
"""
prompt=ChatPromptTemplate.from_template(template)

In [53]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [54]:

rag_chain=(
    {'context': retrieve | (lambda docs : "\n".join( doc.page_content for doc in docs)),'question' : RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [57]:
questions=['list the projects done in meta and teksystems',
           'list the skillset of nivash']

In [58]:
for q in questions:
    print(f"Prompt : {q}")
    answer=rag_chain.invoke(q)
    print(f"Response : {answer}")



Prompt : list the projects done in meta and teksystems
Response : Here are the projects mentioned:

1. LLM Data Pipeline & Annotation Analytics (Meta) - November 2023 to July 2025:
   * Improved data quality and annotation workflows for Generative AI projects.
   * Assisted in process improvement initiatives by analyzing transaction data trends and identifying bottlenecks.

2. Recommendation Updates (POC Project) - Refined AI-generated recommendations using iterative prompt strategies, enhancing accuracy and real-time adaptability for users.

3. Dialogue Flow & Intent Optimization – Internal CX Project (POC Project) - Improved chatbot performance by analyzing and creating intents, reducing fallback responses, and enhancing query accuracy.

Note that the "LLM Data Pipeline & Annotation Analytics" project was done at Meta and the other two projects were part of POC projects at Meta.
Prompt : list the skillset of nivash
Response : Here is the list of skills for NIVASH R:

**Technical Skil